In [1]:
pwd

'/mmfs1/data/aglinska/BC-ABCD-denoise/Code'

In [2]:
!date

Fri Oct 25 12:08:59 EDT 2024


In [3]:
import os
import numpy as np
import pandas as pd
import ants
from tqdm import tqdm
from matplotlib import pyplot as plt

/data/aglinska/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/data/aglinska/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/data/aglinska/anaconda3/lib/python3.8/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
def correlate_columns(arr1, arr2):
    """
    Computes the Pearson correlation between corresponding columns of two matrices.
    
    Parameters:
    arr1 (np.ndarray): First matrix of shape (370, 1000)
    arr2 (np.ndarray): Second matrix of shape (370, 1000)
    
    Returns:
    np.ndarray: 1D array of correlations for each column (size 1000)
    """
    # Ensure input arrays are numpy arrays
    arr1 = np.asarray(arr1)
    arr2 = np.asarray(arr2)
    
    # Subtract the mean of each column (normalize)
    arr1_centered = arr1 - np.mean(arr1, axis=0)
    arr2_centered = arr2 - np.mean(arr2, axis=0)
    
    # Compute the numerator (covariance)
    numerator = np.sum(arr1_centered * arr2_centered, axis=0)
    
    # Compute the denominator (product of standard deviations)
    denominator = np.sqrt(np.sum(arr1_centered**2, axis=0) * np.sum(arr2_centered**2, axis=0))
    
    # Compute the Pearson correlation for each column
    correlation = numerator / denominator
    
    return correlation

# Example usage:
#ARR1 = np.random.rand(370, 1000)  # Replace with your actual matrix
#ARR2 = np.random.rand(370, 1000)  # Replace with your actual matrix
#correlations = correlate_columns(ARR1, ARR2)


def make_RDM(inVec,data_scale='ratio',metric='euclidean'):
    from scipy.spatial.distance import pdist
    from scipy.spatial.distance import squareform
    vec = inVec
    vec = (vec - min(vec.flatten())) / (max(vec.flatten())-min(vec.flatten()))
    
    if np.ndim(inVec)==1: # must be at least 2D
        vec = np.vstack((vec,np.zeros(vec.shape))).transpose()
                   
    mat = squareform(pdist(vec,metric=metric).transpose())
    if data_scale=='ordinal':
        mat[mat!=0]=1 # Make into zeros and ones
        
    return mat

def get_triu(inMat):
    assert np.ndim(inMat)==2, 'not 2 dim, wtf'
    assert inMat.shape[0]==inMat.shape[1], 'not a square'

    n = inMat.shape[0]
    triu_vec = inMat[np.triu_indices(n=n,k=1)]
    return triu_vec

In [5]:
indir = '../Data/020-fmriprepped/'
subs_orig = [s for s in os.listdir(indir) if all((s.startswith('sub'),not s.endswith('.html')))]
n_orig = len(subs_orig)
epi_fn = os.path.join(indir,'{sub}/ses-baselineYear1Arm1/func/{sub}_ses-baselineYear1Arm1_task-nback_run-0{r}_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii')
#cf_fn = os.path.join(indir,'{sub}/ses-baselineYear1Arm1/anat/analysis_mask_CF.nii')
#gm_fn = os.path.join(indir,'{sub}/ses-baselineYear1Arm1/anat/analysis_mask_GM.nii')
cf_fn = os.path.join(indir,'{sub}/ses-baselineYear1Arm1/anat/analysis_mask_CF-strict.nii')
gm_fn = os.path.join(indir,'{sub}/ses-baselineYear1Arm1/anat/analysis_mask_GM-strict.nii')
brain_mask = os.path.join(indir,'{sub}/ses-baselineYear1Arm1/anat/{sub}_ses-baselineYear1Arm1_run-01_space-MNI152NLin2009cAsym_res-2_label-GM_probseg.nii.gz')

subs = [sub for sub in subs_orig if all(( 
    os.path.exists(epi_fn.format(sub=sub,r=1)),
    os.path.exists(epi_fn.format(sub=sub,r=2)),
    os.path.exists(cf_fn.format(sub=sub)),
    os.path.exists(gm_fn.format(sub=sub)),
    os.path.exists(brain_mask.format(sub=sub)),
    ))]

subs.sort()
print(f'{len(subs)}/{len(subs_orig)}')

34/34


In [6]:
has_files = np.array([all([os.path.exists(epi_fn.format(sub=sub,r=1).replace('preproc',f'DEEPCOR-FG-{rep}')) for rep in range(5)]) for sub in subs])
subs = np.array(subs)[has_files]
print(len(subs))

30


In [7]:
ffa = ants.image_read('../Misc/ffa_mask_resampled.nii')
ffa_n = ffa.numpy().sum()

In [8]:
s = 0

In [9]:
sub = subs[s]
print(sub)

sub-NDARINV1H7JEJW1


In [10]:
#compcorr = ants.image_read(os.path.join(indir,'{sub}/ses-baselineYear1Arm1/func/{sub}_ses-baselineYear1Arm1_task-nback_run-0{r}_space-MNI152NLin2009cAsym_res-2_desc-COMPCORR_bold.nii').format(sub=sub,r=r))

In [11]:
#ims = [ants.image_read(epi_fn.format(sub=sub,r=1).replace('preproc','DEEPCOR-FG-{rep}'.format(rep=rep))) for rep in range(5)]

In [12]:
def get_regs(sub,r):
    from nilearn.glm.first_level import make_first_level_design_matrix
    import pandas as pd
    events_fn = f'../Data/010-Data_bids2/{sub}/ses-baselineYear1Arm1/func/{sub}_ses-baselineYear1Arm1_task-nback_run-0{r}_events.tsv'
    events = pd.read_csv(events_fn,delimiter='\t')
    
    t_r = 0.8  # repetition time is 1 second
    n_scans = 370  # the acquisition comprises 128 scans
    frame_times = (np.arange(n_scans) * t_r)
    
    X1 = make_first_level_design_matrix(frame_times,events,drift_model="polynomial",drift_order=3,hrf_model="SPM") #
    
    face_reg = X1[['0_back_negface','0_back_neutface','0_back_place','0_back_posface','2_back_negface','2_back_neutface']].values.sum(axis=1)
    place_reg = X1[['0_back_place','2_back_place']].values.sum(axis=1)
    return face_reg,place_reg

In [13]:
def get_r_map_dff(im,face_reg,place_reg):
    brain_mask = im.sum(axis=-1)>100
    corr_res_face = correlate_columns(im[brain_mask].transpose(),np.array([face_reg for _ in range(int(brain_mask.sum()))]).transpose())
    corr_res_place = correlate_columns(im[brain_mask].transpose(),np.array([place_reg for _ in range(int(brain_mask.sum()))]).transpose())
    corr_brain = np.zeros(brain_mask.flatten().shape)
    corr_brain[brain_mask.flatten()==1] = corr_res_face-corr_res_place
    corr_brain = corr_brain.reshape(ffa.shape)
    return ffa.new_image_like(corr_brain)

In [14]:
for s in tqdm(range(len(subs))):
    for r in [1,2]:
            sub = subs[s]
            compcorr = ants.image_read(epi_fn.format(sub=sub,r=r).replace('preproc','COMPCORR'))
            #ims_tg = [ants.image_read(epi_fn.format(sub=sub,r=1).replace('preproc','DEEPCOR-TG-{rep}'.format(rep=rep))) for rep in range(5)]
            #ims_bg = [ants.image_read(epi_fn.format(sub=sub,r=1).replace('preproc','DEEPCOR-BG-{rep}'.format(rep=rep))) for rep in range(5)]
            ims_fg = [ants.image_read(epi_fn.format(sub=sub,r=r).replace('preproc','DEEPCOR-FG-{rep}'.format(rep=rep))) for rep in range(5)]
            
            #im_tg = np.array([val.numpy() for val in ims_tg]).mean(axis=0)
            #im_bg = np.array([val.numpy() for val in ims_bg]).mean(axis=0)
            im_fg = np.array([val.numpy() for val in ims_fg]).mean(axis=0)
        
            face_reg,place_reg = get_regs(sub,r)

            corr_map_fg = get_r_map_dff(im_fg,face_reg,place_reg)
            corr_map_fg.to_filename(f'../temp/FG-diff-S{s}-R{r}.nii')
        
            corr_map_compcorr = get_r_map_dff(compcorr,face_reg,place_reg)
            corr_map_compcorr.to_filename(f'../temp/COMPCORR-diff-S{s}-R{r}.nii')

  0%|                                                    | 0/30 [00:20<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# fg_run1 = [ants.image_read('../temp/FG-diff-S{s}-R1.nii'.format(s=s)) for s in range(20)]
# fg_run2 = [ants.image_read('../temp/FG-diff-S{s}-R2.nii'.format(s=s)) for s in range(20)]

In [ ]:
# compcorr_run1 = [ants.image_read('../temp/COMPCORR-diff-S{s}-R1.nii'.format(s=s)) for s in range(20)]
# compcorr_run2 = [ants.image_read('../temp/COMPCORR-diff-S{s}-R2.nii'.format(s=s)) for s in range(20)]

In [ ]:
# fg_mean = fg_run1[0].new_image_like(np.array([val.numpy() for val in fg_run1+fg_run2]).mean(axis=0))
# fg_mean.to_filename('../temp/fg-diff-mean.nii')

In [ ]:
# compcorr_mean = compcorr_run1[0].new_image_like(np.array([val.numpy() for val in compcorr_run1+compcorr_run2]).mean(axis=0))
# compcorr_mean.to_filename('../temp/compcorr-diff-mean.nii')

In [ ]:
# (fg_mean.numpy()[ffa.numpy()==1].mean(),compcorr_mean.numpy()[ffa.numpy()==1].mean())

In [ ]:
# np.corrcoef(fg_run1[0].numpy()[ffa.numpy()==1],fg_run1[10].numpy()[ffa.numpy()==1])[0,1]

In [ ]:
# plt.hist([np.corrcoef(fg_run1[i].numpy()[ffa.numpy()==1],fg_run2[i].numpy()[ffa.numpy()==1])[0,1] for i in range(20)])
# plt.hist([np.corrcoef(compcorr_run1[i].numpy()[ffa.numpy()==1],compcorr_run2[i].numpy()[ffa.numpy()==1])[0,1] for i in range(20)])